# 策略迭代(Policy Iteration)

In [4]:
import numpy as np
import pprint
import sys
from lib.envs.gridworld import GridworldEnv

In [5]:
# 載入遊戲
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [6]:
# 策略評估函數
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    policy: [S, A] shaped matrix representing the policy.
    env.P：狀態轉移機率矩陣
    env.P[s][a]：(prob, next_state, reward, done) 陣列
    env.nS：狀態總數
    env.nA：行動總數
    theta: 狀態值函數變動 <= theta，即停止進行。
    discount_factor: 獎勵折扣率(Gamma)
    
    傳回【狀態值函數】
    """
    # 一開始【狀態值函數】均為 0
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # 執行每一種可能的行動
            for a, action_prob in enumerate(policy[s]):
                # 行動可能到達的每一種狀態
                for  prob, next_state, reward, done in env.P[s][a]:
                    # 計算【狀態值函數】期望值，參考圖三. 狀態值函數公式
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # 找出每一個位置的這一輪與上一輪狀態值函數差異最大者
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # 最大差異 < 事先設定值 theta，則停止評估
        if delta < theta:
            break
    return np.array(V)

In [7]:
# 策略改善函數
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    policy_eval_fn: 策略評估函數。
    discount_factor: 獎勵折扣率(Gamma)
        
    Returns:
        傳回 (policy, V) 陣列. 
        - policy：是最佳策略, [S, A] 二為矩陣，含每個狀態/行動組合的機率分配.
        - V：狀態值函數。
        
    """

    # 走下一步的處理，計算行動值函數
    def one_step_lookahead(state, V):
        """
        state: 狀態值
        V: 狀態值函數
        
        傳回【行動值函數】
        """
        A = np.zeros(env.nA)
        # 圖四. 行動值函數公式
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # 隨機策略，每一個位置往上、下、左、右走的機率均為 0.25。
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # 策略評估
        V = policy_eval_fn(policy, env, discount_factor)
        
        # 策略是否趨於穩定的旗標
        policy_stable = True
        
        # 每一個狀態均採貪婪戰術
        for s in range(env.nS):
            # 貪婪戰術
            chosen_a = np.argmax(policy[s])
            
            # 走下一步的處理，計算行動值函數
            action_values = one_step_lookahead(s, V)
            # 找出最大行動值函數
            best_a = np.argmax(action_values)
            
            # 貪婪戰術 不等於 最大行動值函數，繼續迭代
            if chosen_a != best_a:
                policy_stable = False
            # 設定目前狀態下最佳的行動
            policy[s] = np.eye(env.nA)[best_a]
        
        # 策略趨於穩定就結束訓練
        if policy_stable:
            return policy, V

## 測試：顯示策略，即在每個狀態/行動組合的機率分配

In [8]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]



## 顯示狀態值函數

In [9]:
print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



## 顯示最後結果

In [10]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)